<a href="https://colab.research.google.com/github/Abdulraqib20/Vendease-RAG-Application/blob/main/Copy_of_RAG_APP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import pandas as pd;pd.set_option('display.max_columns', None)
import numpy as np
import warnings;warnings.filterwarnings('ignore')

In [47]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [48]:
df = pd.read_csv('/content/drive/MyDrive/Vendease RAG Assessment/AI Engineer Assessment Sample.csv')
df.head()

,flags,instruction,category,intent,response
0,B,question about cancelling order {{Order Number}},ORDER,cancel_order,I've understood you have a question regarding ...
1,BQZ,i have a question about cancelling oorder {{Or...,ORDER,cancel_order,I've been informed that you have a question ab...
2,BLQZ,i need help cancelling puchase {{Order Number}},ORDER,cancel_order,I can sense that you're seeking assistance wit...
3,BL,I need to cancel purchase {{Order Number}},ORDER,cancel_order,I understood that you need assistance with can...
4,BCELN,"I cannot afford this order, cancel purchase {{...",ORDER,cancel_order,I'm sensitive to the fact that you're facing f...


In [49]:
len(df.intent.value_counts())

27

In [50]:
df.isnull().sum()

flags          0
instruction    0
category       0
intent         0
response       0
dtype: int64

In [51]:
for i in df[['category', 'intent']].columns:
  print(df[i].value_counts(), '\n\n')

category
ACCOUNT         5986
ORDER           3988
REFUND          2992
INVOICE         1999
CONTACT         1999
PAYMENT         1998
FEEDBACK        1997
DELIVERY        1994
SHIPPING        1970
SUBSCRIPTION     999
CANCEL           950
Name: count, dtype: int64 


intent
edit_account                1000
switch_account              1000
check_invoice               1000
complaint                   1000
contact_customer_service    1000
delivery_period              999
registration_problems        999
check_payment_methods        999
contact_human_agent          999
payment_issue                999
newsletter_subscription      999
get_invoice                  999
place_order                  998
cancel_order                 998
track_refund                 998
change_order                 997
get_refund                   997
create_account               997
check_refund_policy          997
review                       997
set_up_shipping_address      997
delivery_options             99

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26872 entries, 0 to 26871
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   flags        26872 non-null  object
 1   instruction  26872 non-null  object
 2   category     26872 non-null  object
 3   intent       26872 non-null  object
 4   response     26872 non-null  object
dtypes: object(5)
memory usage: 1.0+ MB


In [53]:
print(df['response'].iloc[:5].tolist())

["I've understood you have a question regarding canceling order {{Order Number}}, and I'm here to provide you with the information you need. Please go ahead and ask your question, and I'll do my best to assist you.", "I've been informed that you have a question about canceling order {{Order Number}}. I'm here to assist you! Please go ahead and let me know what specific question you have, and I'll provide you with all the information and guidance you need. Your satisfaction is my top priority.", "I can sense that you're seeking assistance with canceling your purchase with the purchase number {{Order Number}}. I apologize for any inconvenience caused, and I'm here to guide you through the process.\n\nTo cancel your purchase, please follow these steps:\n\n1. Log into your account: Visit our {{Online Company Portal Info}} and sign in using your credentials.\n2. Locate your order: Once logged in, navigate to the '{{Online Order Interaction}}' or '{{Online Order Interaction}}' section to fin

In [54]:
# # convert to lower case

# df['instruction'] = df['instruction'].str.lower()
# df['category'] = df['category'].str.lower()
# df['intent'] = df['intent'].str.lower()
# df['response'] = df['response'].str.lower()

# df.head()

In [55]:
# # punctuation removal

# import re
# def remove_punctuation(text):
#   return re.sub(r'[^\w\s]', '', text)

# df['instruction'] = df['instruction'].apply(remove_punctuation)
# df['category'] = df['category'].apply(remove_punctuation)
# df['intent'] = df['intent'].apply(remove_punctuation)
# df['response'] = df['response'].apply(remove_punctuation)

# df.head()

In [56]:
# Define a function to apply entity masking by replacing '{{Order Number}}' with a generic token
def mask_entities(text):
    masked_text = text.replace('{{Order Number}}', '[ENTITY_ORDER_NUMBER]')
    return masked_text

df['masked_instruction'] = df['instruction'].apply(mask_entities)
df['masked_response'] = df['response'].apply(mask_entities)
display(df[['masked_instruction', 'masked_response']].head())

,masked_instruction,masked_response
0,question about cancelling order [ENTITY_ORDER_...,I've understood you have a question regarding ...
1,i have a question about cancelling oorder [ENT...,I've been informed that you have a question ab...
2,i need help cancelling puchase [ENTITY_ORDER_N...,I can sense that you're seeking assistance wit...
3,I need to cancel purchase [ENTITY_ORDER_NUMBER],I understood that you need assistance with can...
4,"I cannot afford this order, cancel purchase [E...",I'm sensitive to the fact that you're facing f...


## Function to Apply Entity Masking

This code defines a function called `mask_entities` that replaces occurrences of '{{Order Number}}' in a given text with a generic token '[ENTITY_ORDER_NUMBER]'. It then applies this function to two columns ('instruction' and 'response') in a DataFrame (`df`) and stores the masked versions in new columns 'masked_instruction' and 'masked_response'. Finally, it displays the first few rows of the DataFrame with these masked versions of text.

In [57]:
data = df[['masked_instruction', 'masked_response']]
data.rename(columns={
    'masked_instruction': 'question',
    'masked_response': 'answer'
}, inplace=True)
data.to_csv('data.csv', index=False)

## Renaming Columns and Saving Data to CSV

This code selects the columns 'masked_instruction' and 'masked_response' from the DataFrame `df` and creates a new DataFrame `data` with these columns. It then renames the columns to 'question' and 'answer' respectively. Finally, it saves the DataFrame `data` to a CSV file named 'data.csv', excluding the index.

In [58]:
!pip install -q -U google-generativeai
!pip install gemini-python
!pip install pinecone-client
!pip install openai pinecone-client[grpc] datasets tiktoken langchain gradio
!pip install InstructorEmbedding
!pip install sentence-transformers

!pip install google-cloud-gemini
!pip install langchain-google-genai
!pip install faiss-cpu
!pip install langchain-google-genai

ERROR: Could not find a version that satisfies the requirement google-cloud-gemini (from versions: none)
ERROR: No matching distribution found for google-cloud-gemini


## Installing Required Packages

This code installs several Python packages that will be important in building the RAG App using pip. Here's the list of packages being installed:

1. `google-generativeai` (updated quietly)
2. `gemini-python`
3. `pinecone-client`
4. `openai`
5. `pinecone-client` (with `grpc` extras)
6. `datasets`
7. `tiktoken`
8. `langchain`
9. `gradio`
10. `InstructorEmbedding`
11. `sentence-transformers`
12. `google-cloud-gemini`
13. `langchain-google-genai`
14. `faiss-cpu`

In [63]:
!pip install python-dotenv
!pip install langchain_chroma

## Importing Necessary Libraries

This code imports various libraries necessary for the development of the RAG Application. Here's a breakdown of the imports:

1. `pandas` as `pd`: Library for data manipulation and analysis.
2. `os`: Module for interacting with the operating system.
3. `load_dotenv`: Function to load environment variables from a .env file.
4. `pathlib`: Module for working with file paths.
5. `textwrap`: Module for formatting text.
6. `google.generativeai`: Package related to Google's generative AI capabilities.
7. `openai`: Package for accessing OpenAI's services.
8. `langchain`: Package for developing language-based AI systems.
9. `pinecone`: Package for vector similarity search.
10. `gradio`: Package for creating web-based UIs for machine learning models.

In [64]:
import pandas as pd
import os
from dotenv import load_dotenv

import pathlib
import textwrap
import google.generativeai as genai

import openai
# from openai.embeddings_utils import get_embedding
from langchain.embeddings.openai import OpenAIEmbeddings

from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.agents import Tool
from langchain.agents import initialize_agent

from langchain_text_splitters import CharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_google_genai import GoogleGenerativeAI, ChatGoogleGenerativeAI as Gemini
from langchain.chains import ConversationalRetrievalChain

from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.memory import ConversationBufferMemory

from tqdm.auto import tqdm
from uuid import uuid4

import pinecone
from pinecone import Pinecone, ServerlessSpec
import gradio as gr

In [61]:
# from langchain.vectorstores import Pinecone

In [65]:
data = pd.read_csv('/content/data.csv')
# data.insert(0, 'id_num', range(1, len(data) + 1))
display(data.head(), data.tail())

,question,answer
0,question about cancelling order [ENTITY_ORDER_...,I've understood you have a question regarding ...
1,i have a question about cancelling oorder [ENT...,I've been informed that you have a question ab...
2,i need help cancelling puchase [ENTITY_ORDER_N...,I can sense that you're seeking assistance wit...
3,I need to cancel purchase [ENTITY_ORDER_NUMBER],I understood that you need assistance with can...
4,"I cannot afford this order, cancel purchase [E...",I'm sensitive to the fact that you're facing f...


,question,answer
26867,I am waiting for a rebate of {{Refund Amount}}...,Thank you for sharing your situation regarding...
26868,how to see if there is anything wrong with my ...,Ensuring the accuracy of your restitution is o...
26869,I'm waiting for a reimbjrsement of {{Currency ...,"Firstly, I genuinely understand the importance..."
26870,I don't know what to do to see my reimbursemen...,I've understood you're unsure about how to che...
26871,I need to know if there is anything new on the...,It's completely understandable that you want t...


## Chunking Data Function:

The chunk_data function divides a large dataset into smaller segments, or "chunks." This operation is crucial for efficient data processing, particularly when dealing with extensive datasets that may exceed available memory resources. By segmenting the dataset into manageable chunks, computational tasks such as analysis, transformation, or modeling can be performed iteratively on each segment without overwhelming system resources. The function requires two parameters: the dataset to be chunked (referred to as data) and the desired size of each chunk (chunk_size). It sequentially partitions the dataset into chunks of the specified size, creating separate subsets that can be processed independently. This approach optimizes computational efficiency by distributing the workload and mitigating memory constraints.

In [66]:
# Chunk data

def chunk_data(data: pd.DataFrame, chunk_size: int = 100) -> list[pd.DataFrame]:
    """Groups rows of a DataFrame into chunks of a specified size.

    Args:
        data (pd.DataFrame): The DataFrame to be chunked.
        chunk_size (int): The desired number of rows in each chunk.

    Returns:
        list[pd.DataFrame]: A list of DataFrames, each representing a chunk.
    """
    chunks = []
    num_rows = len(data)
    for i in range(0, num_rows, chunk_size):
        chunks.append(data.iloc[i:i + chunk_size])
    return chunks

chunked_data = chunk_data(data, chunk_size=50)

In [67]:
print(len(chunked_data))

538


## Setting Up Google API Key for Generative AI

This code imports the `userdata` module from the `google.colab` package to retrieve the Google API key stored in the user's environment. It then sets up the API key for Google's Generative AI by using the `configure` function from the `genai` module. This step is necessary for authenticating and authorizing access to Google's Generative AI services, enabling the execution text generation.

In [68]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

**gemini-pro: optimized for text-only prompts.**

## Setting Up Pinecone API Key

This code retrieves the Pinecone API key from the present env using the `userdata` module. It then initializes a Pinecone client object (`pc`) by passing the API key to the `Pinecone` constructor. This step is essential for authenticating and authorizing access to Pinecone's vector similarity search service, allowing for efficient retrieval of similar vectors.

In [69]:
PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')
pc = Pinecone(api_key=PINECONE_API_KEY)

In [70]:
# list pinecone indexes

pc.list_indexes()

{'indexes': [{'dimension': 768,
              'host': 'businesschatbot-4sm66pe.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'businesschatbot',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [71]:
index_name = "businesschatbot"


# pc.create_index(
#     name=index_name,
#     dimension=768,
#     metric="cosine",
#     spec=ServerlessSpec(
#         cloud='aws',
#         region='us-east-1'
#     )
# )

index = pc.Index("businesschatbot")
index, index_name

(<pinecone.data.index.Index at 0x7d4647d1feb0>, 'businesschatbot')

## Embedding Text into Vectors Using the Embedding Model

Now after chunking the texts into smaller segments, the next step is to pass these chunks through an embedding model to obtain their vector representations. The embedding model maps the textual information into high-dimensional vector spaces, where semantic similarities and relationships are preserved.

The `generate_embeddings` function takes a list of DataFrame chunks containing text data and generates embeddings (vector representations) for each document using an embedding model. Here's what it does:

### Function:
- **Name:** generate_embeddings
- **Arguments:**
  - `documents` (list[pd.DataFrame]): A list of DataFrame chunks.
- **Returns:**
  - `list[list[float]]`: A list containing lists of embeddings for each document.

### Description:
The function first initializes an embedding model (`GoogleGenerativeAIEmbeddings`) using a specified model ("models/embedding-001") and the Google API key. It then iterates over each DataFrame chunk in the input list. For each chunk, it combines the 'question' and 'answer' columns into a single text string. Next, it uses the embedding model to generate embeddings for these text strings. The resulting embeddings for each chunk are stored in a list, and this list of embedding lists is returned.

In [32]:
# Embed the Chunked Texts Into Vectors Using the Embedding Model

def generate_embeddings(documents: list[pd.DataFrame]) -> list[list[float]]:
    """
    Generate embeddings for a list of DataFrame chunks.

    Args:
        documents (list[pd.DataFrame]): A list of DataFrame chunks.

    Returns:
        list[list[float]]: A list containing lists of embeddings for each document.
    """
    embed = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GOOGLE_API_KEY)

    embedded = []
    for df_chunk in documents:
        texts_to_embed = df_chunk["question"] + " " + df_chunk["answer"]
        chunk_embeddings = embed.embed_documents(texts_to_embed.tolist())
        embedded.append(chunk_embeddings)

    return embedded

chunked_document_embeddings = generate_embeddings(documents=chunked_data)

# Print embedding dimensions
print(len(chunked_document_embeddings[0]))

50


In [ ]:
def generate_embeddings(documents: list[pd.DataFrame]) -> list[list[float]]:
    """
    Generate embeddings for a list of DataFrame chunks, handling potential embedding service limits.

    Args:
        documents (list[pd.DataFrame]): A list of DataFrame chunks.

    Returns:
        list[list[float]]: A list containing lists of embeddings for each document.
    """
    embed = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GOOGLE_API_KEY)

    embedded = []
    for df_chunk in documents:
      for index, row in df_chunk.iterrows():
        text_to_embed = row["question"] + " " + row["answer"]
        try:
          embedding = embed.embed_documents(text_to_embed)
          embedded.append(embedding)
        except Exception as e:
          print(f"Embedding Error for row {index}: {e}")
          continue

    return embedded

chunked_document_embeddings = generate_embeddings(documents=chunked_data)

In [34]:
print("Length of embeddings list:", len(chunked_document_embeddings))

538

In [ ]:
# without chunking

def generate_embeddings(data: pd.DataFrame) -> list[list[float]]:
    """
    Generate embeddings for all question-answer pairs in a DataFrame.

    Args:
        data (pd.DataFrame): A DataFrame with 'question' and 'answer' columns.

    Returns:
        list[list[float]]: A list of embedding lists, one embedding per question-answer pair.
    """
    embed = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GOOGLE_API_KEY)
    BATCH_SIZE = 50
    embedded = []
    texts_to_embed = []
    for index, row in data.iterrows():
        text_to_embed = row["question"] + " " + row["answer"]
        texts_to_embed.append(text_to_embed)

        # Embed in batches
        if len(texts_to_embed) >= BATCH_SIZE:
            embeddings = embed.embed_documents(texts_to_embed)
            embedded.extend(embeddings)
            texts_to_embed = []  # Reset the batch

    # Embed any remaining texts
    if texts_to_embed:
        embeddings = embed.embed_documents(texts_to_embed)
        embedded.extend(embeddings)

    return embedded

embeddings = generate_embeddings(data)

In [ ]:
#  Upload/Push the Vectors and Text to the Database

def upsert_data_to_pinecone(data_with_metadata: list[dict[str, any]]) -> None:
    """Upserts data with metadata into a Pinecone index.

    Args:
        data_with_metadata (list[dict[str, any]]): List of dictionaries containing 'id', 'values' (embedding), and 'metadata'.
    """
    # Initialize your Pinecone index object (replace placeholders)
    index = pc.Index("businesschatbot")

    # Upsert the data
    index.upsert(vectors=data_with_metadata)

In [ ]:
import hashlib

def generate_short_id(question: str, answer: str) -> str:
    """Generates a short ID based on the question and answer using SHA-256 hash. """
    content = question + answer
    hash_obj = hashlib.sha256()
    hash_obj.update(content.encode("utf-8"))
    return hash_obj.hexdigest()


def combine_vector_and_text(data: pd.DataFrame, embeddings: list[list[float]]) -> list[dict[str, any]]:
    """Combines embeddings, text, and metadata from a DataFrame.

    Args:
        data (pd.DataFrame): DataFrame with 'question' and 'answers' columns.
        embeddings (list[list[float]]): List of lists containing embeddings.

    Returns:
        list[dict[str, any]]: List of dictionaries with 'id', 'values', and 'metadata'.
    """
    if len(data) != len(embeddings):
        raise ValueError("Number of embeddings must match the number of data rows.")

    data_with_metadata = []
    for index, row in data.iterrows():
        question = row["question"]
        answer = row["answer"]
        embedding = embeddings[index]

        doc_id = generate_short_id(question, answer)

        data_item = {
            "id": doc_id,
            "values": embedding,
            "metadata": {
                "question": question,
                "answer": answer
            }
        }
        data_with_metadata.append(data_item)

    return data_with_metadata

data_with_meta_data = combine_vector_and_text(data, embeddings)

In [ ]:
# Combine the Embeddings and the Chunked Text

import hashlib

def generate_short_id(question: str, answer: str) -> str:
    """Generates a short ID based on the question and answer using SHA-256 hash. """
    content = question + answer  # Combine question and answer
    hash_obj = hashlib.sha256()
    hash_obj.update(content.encode("utf-8"))
    return hash_obj.hexdigest()


def combine_vector_and_text(data: pd.DataFrame, embeddings: list[list[float]]) -> list[dict[str, any]]:
    """Combines embeddings, text, and metadata from a DataFrame.

    Args:
        data (pd.DataFrame): DataFrame with 'question' and 'answers' columns.
        embeddings (list[list[float]]): List of lists containing embeddings.

    Returns:
        list[dict[str, any]]: List of dictionaries with 'id', 'values', and 'metadata'.
    """
    data_with_metadata = []

    for index, row in data.iterrows():
        question = row["question"]
        answer = row["answer"]
        embedding = embeddings[index]

        doc_id = generate_short_id(question, answer)

        data_item = {
            "id": doc_id,
            "values": embedding,
            "metadata": {
                "question": question,
                "answer": answer
            }
        }
        data_with_metadata.append(data_item)

    return data_with_metadata

data_with_meta_data = combine_vector_and_text(data, chunked_document_embeddings)

In [41]:
print("Length of DataFrame (number of rows):", len(data))

Length of DataFrame (number of rows): 26872
Length of embeddings list: 538


In [ ]:
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GOOGLE_API_KEY) # Initialize embeddings model
llm = Gemini(model="models/gemini-pro", temperature=0, google_api_key=GOOGLE_API_KEY) # Initialize the Gemini LLM
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True) # Set up conversational memory